In [1]:
print("Hello world!")

Hello world!


### Civilization VI District Optimization Model (Terrain Adjacency Focus)

##### **1. Introduction**
The objective of this model is to determine the optimal placement of city districts within a defined city radius (e.g., 3 tiles, potentially represented by a 7x7 grid/matrix) around a pre-determined City Center tile. The goal is to maximize the total yield generated from adjacency bonuses provided by terrain features, specific resources, and adjacency to the City Center itself. This initial version explicitly excludes adjacency bonuses that districts provide to each other.

##### **2. Data Sets**
$\text{Tiles}$: The set of all tile coordinates (x,y) within the defined city radius.  

$(x_{cc}​,y_{cc}​) \in \text{Tiles}$: The specific coordinates of the pre-determined City Center tile.  

$\text{Tiles}_{\text{Workable}} \subset \text{Tiles}$: The subset of tiles within the radius where districts can potentially be built (i.e., "workable" tiles). This excludes the City Center tile $(x_{cc}, y_{cc})$ and tiles with impassable terrain (like Mountains, unless buildable by specific civs/conditions), or water tiles unless suitable for specific districts (like Harbor).
- Note: The exact definition of $\text{Tiles}_{\text{Workable}}$​ depends on game rules and potential civilization-specific abilities.  

$\text{Districts}$: The set of all possible district types considered (e.g., Campus, Holy Site, Commercial Hub, Harbor, Aqueduct, Entertainment Complex, Water Park, Industrial Zone, etc.).  

$\text{Districts}_{\text{Unique}} \subseteq \text{Districts}$: The subset of districts that are unique per city (i.e., only one of each can be built). Most specialty districts belong here.  

$\text{Districts}_{\text{Mutually Exclusive}}$​: A collection of subsets of $\text{Districts}$, where each subset contains districts that are mutually exclusive with each other.
Example: $\text{Entertainment Complex}, \text{Water Park} \in \text{Districts}_{\text{Mutually Exclusive}}$.  

$\text{Features}$: The set of relevant terrain features, resources, or specific tile types that provide adjacency bonuses (e.g., Mountain, River, Coast, Reef, Geothermal Fissure, Rainforest, Woods, Natural Wonder, Strategic Resource, Luxury Resource, etc.).

#### **3. Input Data**

*These values are pre-determined based on the chosen City Center location and the surrounding map data.*  

$Adj(x,y,x′,y′)$: Binary parameter. 1 if tile $(x,y)\in \text{Tiles}$ and tile $(x′,y′)\in \text{Tiles}$ are adjacent according to the game's hex grid rules, 0 otherwise.  

$HasFeature(x′,y′,f)$: Binary parameter. 1 if tile $(x′,y′)\in \text{Tiles}$ possesses the feature or characteristic $f \in \text{Features}$, 0 otherwise.
- Refinement Note: For features like Rivers, decide if f represents the river tile itself or if you use a parameter like $IsAdjacentToRiver(x,y)$ for buildable tiles next to a river. The latter is often easier for bonus calculation. Let's assume $HasFeature(x′,y′,River)$ means tile $(x′,y′)$ is a river tile.  

$TerrainBonusValue(d,f)$: Numerical parameter. The base yield bonus value that district $d \in \text{Districts}$ receives for being adjacent to one instance of feature $f \in \text{Features}$ on an adjacent tile. (e.g., $TerrainBonusValue(\text{Campus},\text{Mountain})=+1 \text{Science}$). This value should represent the weighted importance if optimizing across multiple yields, or the specific yield value if optimizing for one type (e.g., just Science).  

$CCBonusValue(d)$: Numerical parameter. The base yield bonus value district $d \in \text{District}$ receives for being adjacent to the City Center tile $(x_{cc}​,y_{cc}​)$. (e.g., $CCBonusValue(\text{Commercial Hub})=+1 \text{Gold}$). Often 0 for many districts.  

$IsAdjacentToCC(x,y)$: Binary parameter. 1 if $Adj(x,y,x_{cc}​,y_{cc}​)=1$, 0 otherwise. (Can be pre-calculated).  

$IsAdjacentToFreshwater(x,y)$: Binary parameter. 1 if tile $(x,y)$ is adjacent to a tile $(x′,y′)$ which is a River, Lake, Oasis, or Mountain. (Can be pre-calculated).  

$IsCoast(x,y)$: Binary parameter. 1 if tile $(x,y)$ is a Coast tile.

$IsAdjacentToLand(x,y)$: Binary parameter. 1 if tile $(x,y)$ is adjacent to at least one land tile $(x′,y′)$. (Can be pre-calculated).

#### **4. Decision Variables**

$Build(x,y,d)$: Binary variable.
- $Build(x,y,d)=1$ if district $d \in \text{Districts}$ is built on workable tile $(x,y)\in \text{Tiles}_{\text{Workable}}$​.
- $Build(x,y,d)=0$ otherwise.

#### **5. Objective Function**
Maximize the total weighted adjacency yield derived from adjacent terrain features/resources and adjacency to the City Center.
$$
\begin{gathered}
\text{Max} \quad \sum_{(x,y) \in \text{Tiles}_{\text{Workable}}} \sum_{d \in \text{Districts}} Build(x, y, d) \times \\
 \left( \left( \sum_{(x',y') | Adj(x,y,x',y')} \sum_{f \in Features} HasFeature(x', y', f) \times TerrainBonusValue(d, f) \right) \right. \\
 \left. + IsAdjacentToCC(x,y) \times CCBonusValue(d) \right)
\end{gathered}
$$
Explanation:
- The outer sums iterate through every possible district placement on every workable tile.
- The term contributes to the sum only if $Build(x,y,d)=1$.
- The first inner part $ \sum_{(x',y') | Adj(x,y,x',y')} \dots $ sums the bonuses district $d$ at $(x,y)$ gets from all relevant features f located on all adjacent tiles $(x′,y′)$.
- The second inner part $ IsAdjacentToCC(x,y) \times CCBonusValue(d) $ adds the bonus if the district is placed next to the City Center.

#### **6. Constraints**
**One District Per Workable Tile: Each workable tile can host at most one district.**  
$$ \sum_{d \in \text{Districts}} Build(x, y, d) \leq 1 \quad \forall (x, y) \in \text{Tiles}_{\text{Workable}} $$

**Unique District Limit: Each district type designated as unique can only be built once within the city's radius.**
$$ \sum_{(x,y) \in \text{Tiles}_{\text{Workable}}} Build(x, y, d) \leq 1 \quad \forall d \in \text{Districts}_{\text{Unique}} $$  

**Mutual Exclusivity: For each set MES​et of mutually exclusive districts, at most one district from that set can be built.**
$$ \sum_{d \in \text{Mutually Exclusive}_{\text{Set}}} \sum_{(x,y) \in \text{Tiles}_{\text{Workable}}} Build(x, y, d) \leq 1 \quad \forall \text{Mutually Exclusive}_{\text{Set}} \in \text{Districts}_{\text{Mutually Exclusive}} $$  
###### (Example: For $\text{Mutually Exclusive}_{\text{Set}}=\text{Entertainment Complex},\text{Water Park}$ this becomes: $ \sum_{(x,y) \in \text{Tiles}_{\text{Workable}}} Build(x, y, \text{Entertainment Complex}) + \sum_{(x,y) \in \text{Tiles}_{\text{Workable}}} Build(x, y, \text{Water Park}) \leq 1 $)

**Aqueduct Placement Requirements: An Aqueduct can only be built if the tile (x,y) is adjacent to the City Center AND adjacent to a freshwater source.**
$$ Build(x, y, \text{Aqueduct}) \leq IsAdjacentToCC(x, y) \quad \forall (x, y) \in \text{Tiles}_{\text{Workable}} $$
$$ Build(x, y, \text{Aqueduct}) \leq IsAdjacentToFreshwater(x, y) \quad \forall (x, y) \in \text{Tiles}_{\text{Workable}} $$  

###### (These constraints force $Build(x,y,\text{Aqueduct})$ to be 0 if the conditions aren't met. The tile $(x,y)$ must also be in $\text{Tiles}_{\text{Workable}}$​).  

**Harbor Placement Requirements: A Harbor can only be built if the tile (x,y) is a Coast tile AND is adjacent to a land tile.**   

$$ Build(x, y, \text{Harbor}) \leq IsCoast(x, y) \quad \forall (x, y) \in \text{Tiles}_{\text{Workable}} $$
$$ Build(x, y, \text{Harbor}) \leq IsAdjacentToLand(x, y) \quad \forall (x, y) \in \text{Tiles}_{\text{Workable}} $$
###### (The tile $(x,y)$ must also be in $\text{Tiles}_{\text{Workable}}$​, meaning coast tiles suitable for harbors must be considered workable).  

###### (Optional) Add Other Specific Placement Rules: Include similar constraints for districts like Dams, Canals, etc., based on their specific geographical requirements (e.g., must be on a river tile, must span two landmasses, etc.).


